In [28]:
import sqlite3
import pandas as pd

# Create your connection.
# DL : https://drive.google.com/file/d/10CtW_gZtNHQbQQ2A7QRiXXTh4obXHOQk/view?usp=sharing
con = sqlite3.connect('batch_1.db')
cursorObj = con.cursor()
cursorObj.execute('SELECT name from sqlite_master where type= "table"')
print(cursorObj.fetchall())

[('aeroports',), ('compagnies',), ('vols',)]


In [29]:
aeroports = pd.read_sql_query("SELECT * FROM aeroports", con)
aeroports = aeroports.rename(columns={
    'CODE IATA': 'CODE_IATA',
    'PRIX RETARD PREMIERE 10 MINUTES':'PRIX_RETARD_10',
    'PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES':'PRIX_RETARD_APRES'
})
aeroports.to_csv("aeroports.csv", index=False)
aeroports.head()

,CODE_IATA,NOM,LIEU,PAYS,LONGITUDE,LATITUDE,HAUTEUR,PRIX_RETARD_10,PRIX_RETARD_APRES
0,MCT,Muscat International Airport,Muscat,OM,58.284400939941406,23.593299865722656,48.0,6695,361
1,SOU,Southampton Airport,Southampton,GB,-1.3567999601364136,50.95029830932617,44.0,1061,71
2,PNH,Phnom Penh International Airport,Phnom Penh,KH,104.84400177001953,11.546600341796875,40.0,2402,909
3,BLR,Kempegowda International Airport,Bangalore,IN,77.706299,13.1979,3000.0,8641,550
4,FFD,RAF Fairford,Fairford,GB,-1.7900300025900002,51.6822013855,286.0,6883,870


In [30]:
compagnies = pd.read_sql_query("SELECT * FROM compagnies", con)
compagnies.to_csv("compagnies.csv", index=False)

In [31]:
vols = pd.read_sql_query("SELECT * FROM vols", con)
vols = vols.rename(columns={
'CODE AVION':'CODE_AVION',
'AEROPORT DEPART':'AEROPORT_DEPART',
'AEROPORT ARRIVEE':'AEROPORT_ARRIVEE',
'DEPART PROGRAMME':'HEURE_DEPART_PROGRAMME',
'HEURE DE DEPART':'HEURE_DEPART',
'RETART DE DEPART':'TEMPS_RETARD_DEPART',
'TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE':'TEMPS_DEPLACEMENT_DECOLLAGE',
'DECOLLAGE':'HEURE_DECOLLAGE',
'TEMPS PROGRAMME':'TEMPS_PROGRAMME',
'TEMPS PASSE':'TEMPS_PASSE',
'TEMPS DE VOL':'TEMPS_VOL',
'DISTANCE':'DISTANCE',
'ATTERRISSAGE':'HEURE_ATTERRISSAGE',
"TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE":'TEMPS_DEPLACEMENT_ATTERRISSAGE',
'ARRIVEE PROGRAMMEE':'HEURE_ARRIVEE_PROGRAMMEE',
"HEURE D'ARRIVEE":'HEURE_ARRIVEE',
"RETARD A L'ARRIVEE":'TEMPS_RETARD_ARRIVEE',
'DETOURNEMENT':'DETOURNEMENT',
'ANNULATION':'ANNULATION',
'DATE':'DATE',
'NIVEAU DE SECURITE':'NIVEAU_SECURITE',
#'COMPAGNIE AERIENNE':'COMPAGNIE',
"RAISON D'ANNULATION":"RAISON_ANNULATION",
"RETARD SYSTEM":"RETARD_SYSTEM",
"RETARD SECURITE":"RETARD_SECURITE",
"RETARD COMPAGNIE":"RETARD_COMPAGNIE",
"RETARD AVION":"RETARD_AVION",
"RETARD METEO":"RETARD_METEO"
})
vols.to_csv("vols.csv", index=False)

In [32]:
aeroports = aeroports.drop_duplicates(subset=['CODE_IATA'])
airports_start = aeroports.add_prefix('DEPART_')
airports_end = aeroports.add_prefix('ARRIVEE_')

# Add companies
vols_and_companies = vols.join(compagnies.set_index(['CODE']),
                               on=['COMPAGNIE AERIENNE'],
                               how='left')
print(vols_and_companies.shape)

(3000000, 31)


In [33]:
# Add airport departure information
vols_and_companies_departure = vols_and_companies.join(airports_start.set_index(
    ['DEPART_CODE_IATA']),
    on=['AEROPORT_DEPART'],
    how='inner')

In [34]:
# Add airport arrival information
full_df = vols_and_companies_departure.join(airports_end.set_index(
    ['ARRIVEE_CODE_IATA']),
    on=['AEROPORT_DEPART'],
    how='inner')

In [35]:
full_df.to_csv("full_join.csv", index=False)